# Smart Retail: Customer Purchase Prediction

This notebook implements an end-to-end demo recommendation/prediction pipeline:
1. Generate **synthetic** customers, products, and transactions  
2. Perform quick EDA  
3. Feature engineering (RFM + spending behavior)  
4. Customer segmentation using **K-Means**  
5. Next purchase category prediction using **Random Forest**  
6. Output top-k category recommendations with probabilities (CSV)

(Aligned with the project report.) fileciteturn0file0


In [ ]:
# Core libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Reproducibility
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)


## 1) Generate synthetic retail data

In [ ]:
from datetime import datetime, timedelta

# Parameters (adjust to scale up/down)
N_CUSTOMERS = 500
N_PRODUCTS  = 200
N_TXNS      = 8000
N_CATEGORIES = 8

categories = ["fashion","electronics","grocery","home","beauty","sports","toys","books"][:N_CATEGORIES]

# Customers
customers = pd.DataFrame({
    "customer_id": np.arange(1, N_CUSTOMERS+1),
    "age": rng.integers(18, 70, size=N_CUSTOMERS),
    "gender": rng.choice(["F","M","Other"], size=N_CUSTOMERS, p=[0.48,0.48,0.04]),
    "income_band": rng.choice(["low","mid","high"], size=N_CUSTOMERS, p=[0.35,0.50,0.15]),
    "region": rng.choice(["west","midwest","south","northeast"], size=N_CUSTOMERS, p=[0.25,0.20,0.30,0.25]),
})

# Products
products = pd.DataFrame({
    "product_id": np.arange(1, N_PRODUCTS+1),
    "category": rng.choice(categories, size=N_PRODUCTS, p=np.ones(len(categories))/len(categories)),
})

# Category price ranges (roughly)
cat_price = {
    "fashion": (15, 120),
    "electronics": (60, 900),
    "grocery": (2, 50),
    "home": (10, 300),
    "beauty": (5, 80),
    "sports": (10, 250),
    "toys": (5, 120),
    "books": (5, 60),
}
def sample_price(cat, size=1):
    low, high = cat_price.get(cat, (5, 100))
    # right-skewed with lognormal then clipped
    x = rng.lognormal(mean=np.log((low+high)/2), sigma=0.6, size=size)
    return np.clip(x, low, high)

# Create transaction timestamps with light seasonality
end_date = datetime(2025, 12, 31)
start_date = end_date - timedelta(days=365)

def sample_date(size=1):
    # day-of-year weighting (higher in Nov-Dec)
    days = np.arange(365)
    weights = np.ones(365)
    weights[300:365] *= 1.6  # holiday bump
    weights[0:31] *= 1.1     # January bump
    weights = weights / weights.sum()
    chosen = rng.choice(days, size=size, p=weights)
    return [start_date + timedelta(days=int(d), hours=int(rng.integers(0,24))) for d in chosen]

# Base category preference per customer (simple: depends on income_band + randomness)
pref = {}
for cid, inc in zip(customers.customer_id, customers.income_band):
    base = np.ones(len(categories))
    if inc == "low":
        boost = {"grocery": 1.6, "books":1.2, "beauty":1.1}
    elif inc == "mid":
        boost = {"fashion":1.3, "home":1.2, "grocery":1.2}
    else:
        boost = {"electronics":1.5, "fashion":1.2, "home":1.2}
    for i, c in enumerate(categories):
        base[i] *= boost.get(c, 1.0)
    # add noise
    base = base * rng.uniform(0.8, 1.2, size=len(categories))
    pref[cid] = base / base.sum()

# Transactions
tx_customer = rng.integers(1, N_CUSTOMERS+1, size=N_TXNS)
tx_dates = sample_date(N_TXNS)

tx_rows = []
for i in range(N_TXNS):
    cid = int(tx_customer[i])
    cat = rng.choice(categories, p=pref[cid])
    # pick a product from that category
    prod_id = int(products.loc[products.category == cat, "product_id"].sample(
        n=1, random_state=int(rng.integers(0, 1_000_000_000))
    ).iloc[0])
    price = float(sample_price(cat, 1)[0])
    qty = int(rng.integers(1, 4))
    tx_rows.append((i+1, cid, prod_id, cat, tx_dates[i], qty, price))

transactions = pd.DataFrame(tx_rows, columns=["txn_id","customer_id","product_id","category","txn_time","qty","unit_price"])
transactions["amount"] = transactions["qty"] * transactions["unit_price"]

customers.head(), products.head(), transactions.head()


## 2) Quick EDA

In [ ]:
print("Customers:", customers.shape)
print("Products:", products.shape)
print("Transactions:", transactions.shape)

# Category distribution
cat_counts = transactions["category"].value_counts().sort_values(ascending=False)
display(cat_counts)

plt.figure()
cat_counts.plot(kind="bar")
plt.title("Transaction count by category")
plt.xlabel("Category")
plt.ylabel("Count")
plt.show()

# Spend distribution
plt.figure()
transactions["amount"].clip(upper=transactions["amount"].quantile(0.99)).hist(bins=40)
plt.title("Transaction amount distribution (clipped at 99th pct)")
plt.xlabel("Amount")
plt.ylabel("Frequency")
plt.show()


## 3) Feature engineering (RFM + behavior)

In [ ]:
# RFM features at customer level
snapshot_date = transactions["txn_time"].max() + pd.Timedelta(days=1)

rfm = (transactions
       .groupby("customer_id")
       .agg(
           recency_days=("txn_time", lambda x: (snapshot_date - x.max()).days),
           frequency=("txn_id", "count"),
           monetary=("amount", "sum"),
           avg_basket=("amount", "mean"),
           avg_qty=("qty", "mean"),
       )
       .reset_index())

# Category shares per customer
cat_pivot = (transactions
             .pivot_table(index="customer_id", columns="category", values="amount", aggfunc="sum", fill_value=0.0))
cat_share = cat_pivot.div(cat_pivot.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
cat_share.columns = [f"share_{c}" for c in cat_share.columns]

features = (customers
            .merge(rfm, on="customer_id", how="left")
            .merge(cat_share.reset_index(), on="customer_id", how="left"))

features = features.fillna(0)
features.head()


## 4) Customer segmentation with K-Means

In [ ]:
# Select numeric columns for clustering (exclude IDs)
num_cols = ["age","recency_days","frequency","monetary","avg_basket","avg_qty"] + [c for c in features.columns if c.startswith("share_")]
X_cluster = features[num_cols].copy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# Choose k (simple fixed k; feel free to tune)
K = 5
kmeans = KMeans(n_clusters=K, n_init=10, random_state=RANDOM_SEED)
features["segment"] = kmeans.fit_predict(X_scaled)

features["segment"].value_counts().sort_index()


## 5) Next purchase category prediction (Random Forest)

In [ ]:
# Build a supervised dataset:
# For each customer, take their last transaction category as the label,
# and use features computed at snapshot_date (works for demo).

last_tx = (transactions.sort_values("txn_time")
           .groupby("customer_id")
           .tail(1)[["customer_id","category","txn_time"]]
           .rename(columns={"category":"label_next_category","txn_time":"last_tx_time"}))

data = features.merge(last_tx, on="customer_id", how="inner")

# Encode categoricals
model_df = data.copy()
model_df = pd.get_dummies(model_df, columns=["gender","income_band","region"], drop_first=True)

y = model_df["label_next_category"]
X = model_df.drop(columns=["label_next_category","last_tx_time"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_SEED, stratify=y
)

rf = RandomForestClassifier(
    n_estimators=300,
    random_state=RANDOM_SEED,
    class_weight="balanced_subsample",
    n_jobs=-1
)
rf.fit(X_train, y_train)

pred = rf.predict(X_test)
print(classification_report(y_test, pred))


## 6) Top-k recommendations with probabilities + export CSV

In [ ]:
from pathlib import Path

def top_k_recommendations(model, X_input, k=3):
    proba = model.predict_proba(X_input)
    classes = model.classes_
    top_idx = np.argsort(proba, axis=1)[:, ::-1][:, :k]
    top_classes = classes[top_idx]
    top_scores = np.take_along_axis(proba, top_idx, axis=1)
    return top_classes, top_scores

# Generate recommendations for all customers in data
X_all = X
top_k = 3
top_cats, top_scores = top_k_recommendations(rf, X_all, k=top_k)

out = pd.DataFrame({
    "customer_id": model_df["customer_id"].values,
    "segment": data["segment"].values,
    "actual_last_category": data["label_next_category"].values,
})

for i in range(top_k):
    out[f"rec_{i+1}_category"] = top_cats[:, i]
    out[f"rec_{i+1}_score"] = np.round(top_scores[:, i], 4)

# Save
outputs_dir = Path("../outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)
csv_path = outputs_dir / "predictions.csv"
out.to_csv(csv_path, index=False)

out.head(10), str(csv_path)


## 7) Example: recommendations for a specific customer

In [ ]:
def recommend_for_customer(customer_id: int, k=2):
    row_idx = np.where(model_df["customer_id"].values == customer_id)[0]
    if len(row_idx) == 0:
        raise ValueError("customer_id not found in modeling dataset")
    i = int(row_idx[0])
    recs = [(out.loc[i, f"rec_{j}_category"], float(out.loc[i, f"rec_{j}_score"])) for j in range(1, k+1)]
    return recs

customer_id = 10
print("Customer", customer_id, "segment:", int(out.loc[out.customer_id==customer_id, "segment"].iloc[0]))
print("Top-2 recommendations:", recommend_for_customer(customer_id, k=2))
